<a href="https://colab.research.google.com/github/kcw0331/Deeplearning/blob/main/5_1_introduction_to_convnets(%EA%B9%80%EC%B0%BD%EC%9A%B0).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 먼저 tensorflow를 먼저 불러와 준다.

In [ ]:
import tensorflow
tensorflow.keras.__version__

'2.4.0'

# 5.1 - Introduction to convnets

This notebook contains the code sample found in Chapter 5, Section 1 of [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python?a_aid=keras&a_bid=76564dff). Note that the original text features far more content, in particular further explanations and figures: in this notebook, you will only find source code and related comments.

----

First, let's take a practical look at a very simple convnet example. We will use our convnet to classify MNIST digits, a task that you've already been 
through in Chapter 2, using a densely-connected network (our test accuracy then was 97.8%). Even though our convnet will be very basic, its 
accuracy will still blow out of the water that of the densely-connected model from Chapter 2.

The 6 lines of code below show you what a basic convnet looks like. It's a stack of `Conv2D` and `MaxPooling2D` layers. We'll see in a 
minute what they do concretely.
Importantly, a convnet takes as input tensors of shape `(image_height, image_width, image_channels)` (not including the batch dimension). 
In our case, we will configure our convnet to process inputs of size `(28, 28, 1)`, which is the format of MNIST images. We do this via 
passing the argument `input_shape=(28, 28, 1)` to our first layer.

##### 이제 여기에서는 챕터 5에서 배울 예제를 다룬다. 그리고 여기에서는 챕터2에서 봤던 MNIST 셋에서는 dense 레이어를 사용해서 accuracy가 97.8%까지 나왔는데, 여기에서는 MNIST 셋에 Convnet을 3개 사용하여 얼마나 accuracy를 향상 시킬 수 있을지 확인한다.

In [ ]:
#여기에서는 convnet을 먼저 돌려보는 것 부터 시작하고 있다.
from tensorflow.keras import layers
from tensorflow.keras import models

model = models.Sequential()  #모형을 Sequential로 달아 두었다. 
model.add(layers.Conv2D(32, (3, 3), padding = "valid", activation='relu', input_shape=(28, 28, 1))) #그리고 여기에서 convnet 레이어를 3개를 달아 주었다.
model.add(layers.MaxPooling2D((2, 2))) #그리고 convnet 가운데에 MaxPooling을 통해서 풀링읗 해준다. 자세한 설명은 다음 슬라이드에서 설명하신다고 하심.
model.add(layers.Conv2D(64, (3, 3),  padding = "valid", activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3),  padding = "valid", activation='relu'))


In [ ]:
# 이걸 사용해서 아래 있는 코드들을 동일하게 실행하게되면 accuracy가 98%이 나오게 된다.
from tensorflow.keras import layers
from tensorflow.keras import models

model = models.Sequential()   
model.add(layers.Conv2D(10, (5, 5), padding = "valid", activation='relu', input_shape=(28, 28, 1))) 
model.add(layers.MaxPooling2D((2, 2))) 
model.add(layers.Conv2D(20, (5, 5),  padding = "valid", activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))


Let's display the architecture of our convnet so far:

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


##### **model.summary()를 돌려준 결과 output이 3 콤마 3콤마 64 차원의 데이터로 나왔다.**




You can see above that the output of every `Conv2D` and `MaxPooling2D` layer is a 3D tensor of shape `(height, width, channels)`. The width 
and height dimensions tend to shrink as we go deeper in the network. The number of channels is controlled by the first argument passed to 
the `Conv2D` layers (e.g. 32 or 64).

The next step would be to feed our last output tensor (of shape `(3, 3, 64)`) into a densely-connected classifier network like those you are 
already familiar with: a stack of `Dense` layers. These classifiers process vectors, which are 1D, whereas our current output is a 3D tensor. 
So first, we will have to flatten our 3D outputs to 1D, and then add a few `Dense` layers on top:

- 여기에서는 3콤마 3콤마 64차원을 Flatten시켜주고 Dense=30, Dense=10을 만들어 준다.

In [ ]:
model.add(layers.Flatten())  
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dense(10, activation='softmax')) #그래서 이코드에서는 0~9까지 classification 할 수 있는 그런식으로 모형을 구성한다.

We are going to do 10-way classification, so we use a final layer with 10 outputs and a softmax activation. Now here's what our network 
looks like:

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 30)                1

- 위에 있는 코드를 실행하고 나서 마지막에 Dense 10은 우리가 0~9까지의 손으로 쓴 글씨를 모형화 해주는 모델이 된다.

As you can see, our `(3, 3, 64)` outputs were flattened into vectors of shape `(576,)`, before going through two `Dense` layers.

Now, let's train our convnet on the MNIST digits. We will reuse a lot of the code we have already covered in the MNIST example from Chapter 
2.

In [ ]:
from tensorflow.keras.datasets import mnist  #MNIST데이터를 사용하기 위해서 MNIST데이터를 읽어 와주고rom tensorflow.keras.datasets import mnist  #MNIST데이터를 사용하기 위해서 MNIST데이터를 읽어 와주고
from tensorflow.keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()


11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
train_images.shape

(60000, 28, 28)

In [ ]:
import numpy as np  #그리고 이게 0에서 255사이의 값을 갖는다.그래서 max를 하게 되면 255의 값이 나오게 된다. 
np.max(train_images)

255

In [ ]:
#여기에서는 MNIST데이터를 프리 프로세싱 시켜준다.
#train_images.shape을 하게 되면 (60000, 28, 28)이 나오게 되는데 우리는 1을 넣어주기 위해서 reshape을 해준다.
train_images = train_images.reshape((60000, 28, 28, 1))  #여기에서는 28이 가로, 28이 세로, 그리고 1은 채널이고 원래 RGB는 3인데, 여기에서는 흑백이라서 1이 된다. 
train_images = train_images.astype('float32') / 255 #여기에서는 255로 나누어서 0에서 1의 값이 되도록 표준화를 시켜준다. 

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

#그래서 여기에서는 train set과 test set을 표준화 시켜준다. 

In [ ]:
train_labels[:5]  #train_labels를 5개 까지 하게 되면 5, 0, 4, 1, 9라는 값이 나오게 된다. 

array([5, 0, 4, 1, 9], dtype=uint8)

In [ ]:
train_labels = to_categorical(train_labels)  #그리고 one_hot_encoding을 사용해서 categorical로 바꾸어 주게된다.
test_labels = to_categorical(test_labels)

In [ ]:
train_labels[:5]  #그래서 one_hot_encoding을 하고 나서 train_labels을 하게 되면 5는 0, 0, 0, 0, 1, 0, 0, 0, 0 그리고 0은 1, 0, 0, 0, 0, 0, 0, 0, 0이 된다.

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

In [ ]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',  #그리고 compile을 categorical_crossentropy를 해준다.
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
938/938 [==============================] - 36s 4ms/step - loss: 0.4589 - accuracy: 0.8568
Epoch 2/5
938/938 [==============================] - 4s 4ms/step - loss: 0.0556 - accuracy: 0.9820
Epoch 3/5
938/938 [==============================] - 3s 4ms/step - loss: 0.0353 - accuracy: 0.9890
Epoch 4/5
938/938 [==============================] - 3s 4ms/step - loss: 0.0253 - accuracy: 0.9920
Epoch 5/5
938/938 [==============================] - 3s 4ms/step - loss: 0.0202 - accuracy: 0.9935


Let's evaluate the model on the test data:

In [ ]:
#여기에서는 training을 하고나서 test_loss와 test_acc를 확인해준다.
test_loss, test_acc = model.evaluate(test_images, test_labels)

313/313 [==============================] - 1s 3ms/step - loss: 0.0258 - accuracy: 0.9923


In [ ]:
#그리고 여기에서는 test_acc를 출력해준다.
test_acc

0.9922999739646912

# 그래서 코드를 돌려본 결과 99.3%이라는 결과가 나오는 것을 확인할 수 있다.

While our densely-connected network from Chapter 2 had a test accuracy of 97.8%, our basic convnet has a test accuracy of 99.3%: we 
decreased our error rate by 68% (relative). Not bad! 